# Free Music Alternative Playlists
Create your playlists! :D

In [1]:
import pickle
import random
import numpy as np
import pandas as pd
import networkx as nx
from pygsp import graphs, filters
from scipy.spatial import cKDTree

# Interactive output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Parameters

In [2]:
N = 10;  # songs in the playlist

tau = 85 # heat diffusion coefficient in the range [20:100]
         # This parameter corresponds to the spreading of the exploration
         # in case you use method 2. Higher tau => more variety

# Data Loading

In [3]:
# Load data
dataSet = pd.read_pickle('data/dataSet.csv')
dfmax=np.load('data/dfmax.npy')
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
tracks_raw = pd.read_csv('data/raw_tracks.csv', index_col=0);

In [4]:
# Load custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

features_c=features_c.divide(dfmax)
numb_cust=len(features_c)
# Uncomment to see the extracted features
#features_c.head()

# Functions
Functions used, primarily for displaying the playlists

In [5]:
# Playlists
meta_playlist = dict()

# Dictionary to link the node position and the track ID
idx_dict={i : dataSet.index[i] for i in range(len(dataSet.index))}  
inv_dict = {v: k for k, v in idx_dict.items()}

In [6]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, 'Open')+'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp'+'<a target="_blank" href="{}">{}</a>'.format(val+'/download', 'Download')

In [7]:
def display_result(playlist):
    result=pd.DataFrame({'Artist': tracks.artist.name[playlist], \
                         'Title':tracks.track.title[playlist], \
                         'Links':tracks_raw['track_url'][playlist]}) 

    # Output 
    result.reset_index(inplace=True)
    result.drop(columns='track_id',inplace=True)
    result = result.style.format({'Links':make_clickable})
    
    # Output
    display(result)

# 518D

In [8]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])

# Construct the KD Tree
tree = cKDTree(points_ref)

In [9]:
#list of the tracks nearest to each custom song
nearest_tracks=dict()

# sub-meta playlist
over_playlist = list()

# For each Playlist we find the nearest point in the KD-Tree with the given songs
for name in tid_c['Playlist name'].unique():
    tid = np.array(tid_c[tid_c['Playlist name'] == name].index.values)
    
    # For each features we extract the track id for the corresponding playlist
    features_c_t = np.array([features_c[features_c.index == idx].values[0] for idx in tid])
    
    # Find the 10th nearest points with Euclidean norm in the tree
    dist, idx = tree.query(features_c_t, k=N, p=2)
    
    # We get the correspond Track ID from the position IDX
    playlist = list([idx_dict[node] for node in idx.ravel()])
    
    # For each song in the playlist we get the best match and store it for later use
    top_tracks = [idx_dict[x] for x in idx[:,0]]
    nearest_tracks.update({name:top_tracks})
    
    # Select in the closest neighbours N random tracks
    playlist = random.sample(playlist,N)
    
    # Add to sub-meta playlist
    over_playlist.append(playlist)

#
meta_playlist.update({'518D':over_playlist})

## 2-D coords based

In [10]:
# Create the KD-Tree from the graphs coordinates
coords2D = np.load("data/coords2D.npy").item()

graphCoords=list()
for i in coords2D:
    graphCoords.append(tuple(coords2D[i]))

graphCoords=np.array(graphCoords)
graphTree = cKDTree(graphCoords)

In [11]:
over_playlist = list()

# For each playlist, access nearest tracks
for name in tid_c['Playlist name'].unique():
    
    playlist=list()
    # Retrieve the closest track ID for the given playlist
    tid = nearest_tracks[name]
    
    # Find closest tracks in the graph for a given custom song
    dist, idx = graphTree.query([tuple(coords2D[x]) for x in tid], k=N, p=2)
    
    # Create the playlist
    playlist = list([idx_dict[node] for node in idx.ravel()])

    # Select in the closest neighbours N random tracks
    playlist=random.sample(playlist,N)

    # Add to sub-meta playlist
    over_playlist.append(playlist)
    
#
meta_playlist.update({'2D':over_playlist})

## Heat Music

In [12]:
# Construct Graph
conn_graph = np.load("data/connGraph.npy")
G = graphs.Graph(conn_graph)
G.compute_fourier_basis()
G.set_coordinates()

In [13]:
over_playlist = list()

# For each playlist, access nearest tracks
for name in tid_c['Playlist name'].unique():

    # Map the nearest track IDs to the nodes position
    musics_pos = [inv_dict[tid] for tid in nearest_tracks[name]]

    list_heat_neighbour = np.array([]).astype(int)
    s_out_moy = np.zeros(G.N)

    for i in musics_pos:
        s = np.zeros(G.N)
        s[i]=1

        g = filters.Heat(G, tau, normalize=False)
        s_out = g.filter(s, method='chebyshev')
        s_out = s_out/max(s_out)

        s_out_moy = s_out_moy + s_out

    ind_max_heat = np.argsort(s_out_moy)
    list_heat_neighbour= np.append(list_heat_neighbour,ind_max_heat[-N:])
    
    # Create the playlist
    playlist = [idx_dict[idx] for idx in list_heat_neighbour]
    
    # Select in the closest neighbours N random tracks
    playlist = random.sample(playlist,N)

    # Add to sub-meta playlist
    over_playlist.append(playlist)
    
# 
meta_playlist.update({'Heat':over_playlist})

# Output

In [14]:
def method_selection(m):

    for i,playlist in enumerate(tid_c['Playlist name'].unique()):
        print('Playlist:', playlist)
        display_result(meta_playlist[m][i])

In [15]:
# '518D' -> 518D features space | closely related songs, tonality and rythm are well respected, more genre mixity
# '2D' -> 2D graphs coords      | closely related, but staying within the same genre
# 'Heat' -> Heat diffusion      | more variety, depending on the tau parameter
interact(method_selection, m=['518D','2D', 'Heat']);

interactive(children=(Dropdown(description='m', options=('518D', '2D', 'Heat'), value='518D'), Output()), _dom…